In [1]:
# -*- coding: utf-8 -*-
import re
import csv
import pandas as pd
from pathlib import Path

# 大量の条文データを含むCSVファイルを読み込む
csv_path = "/home/ubuntu/cur/isep/main2.2.csv"
df = pd.read_csv(csv_path)

print(f"データ形状: {df.shape}")
print(f"カラム: {df.columns.tolist()}")
print(f"最初の数行:\n{df.head(3)}")

# 「本文」カラムからすべてのテキストを結合
texts = "\n".join(df["本文"].dropna().astype(str))

print(f"\n結合後のテキスト長: {len(texts):,} 文字")
print(f"処理行数: {len(df):,} 行")

データ形状: (102, 4)
カラム: ['本文', '制定年', '自治体', '区分']
最初の数行:
                                                  本文   制定年  自治体    区分
0  ○大空町再生可能エネルギー発電施設の設置に関する条例\r\n令和7年3月12日\r\n条例第...  2025  大空町    条例
1  ○大空町再生可能エネルギー発電施設の設置に関する条例施行規則\r\n令和7年3月27日\r\...  2025  大空町  施行規則
2  ○羅臼町再生可能エネルギー推進条例施行規則\r\n令和7年3月13日\r\n規則第2号\r\...  2025  羅臼町  施行規則

結合後のテキスト長: 523,040 文字
処理行数: 102 行


In [2]:
# 大量の条例データから候補用語を抽出するための強化版パターン

import unicodedata

def extract_candidates(text):
    # Normalize
    text = unicodedata.normalize("NFKC", text)
    
    # 拡張された接尾辞パターン - 条例に頻出する行政・法律用語
    suffixes = r"(区域|地域|地区|区域等|区域内|区域外|区域図|地域内|地域外|地区内|" \
               r"許可|認可|承認|同意|届出|届|申請|申請書|報告|報告書|通知|通知書|" \
               r"証明|証明書|明細|明細書|計画|計画書|台帳|図書|" \
               r"基準|規則|手続|手続き|協議|事前協議|事後協議|" \
               r"説明|説明会|周知|公告|公表|" \
               r"標識|看板|" \
               r"撤去|除去|解体|廃止|廃棄|処分|" \
               r"維持|維持管理|管理|保守|点検|" \
               r"原状回復|復旧|" \
               r"措置|対策|防止|抑制|" \
               r"構造|構造図|平面図|断面図|配置図|位置図|現況図|" \
               r"写真|現況写真|" \
               r"資金計画|資金計画書|工程表|工事工程表|" \
               r"意見|意見書|要望|協定|あっせん|調停|" \
               r"保全|保護|保存|指定|指定地|" \
               r"市街化調整区域|市街化区域|" \
               r"抑制区域|禁止区域|規制区域|保全区域|特別保全区域|保護区域|" \
               r"保安林|国有林|民有林|" \
               r"土砂災害特別警戒区域|土砂災害警戒区域|" \
               r"急傾斜地崩壊危険区域|地すべり防止区域|" \
               r"農用地|農用地区域|農地|農業振興地域|" \
               r"砂防指定地|砂防設備|" \
               r"自然公園|国立公園|国定公園|県立自然公園|" \
               r"特別地域|特別保護地区|普通地域|" \
               r"河川区域|河川保全区域|河川予定地|海岸区域|海岸保全区域|港湾区域|" \
               r"景観計画区域|景観地区|風致地区|" \
               r"文化財|史跡|名勝|天然記念物|埋蔵文化財|伝統的建造物群保存地区|" \
               r"鳥獣保護区|特別保護地区|狩猟区|" \
               r"特定保全種|希少野生動植物|" \
               r"太陽光発電設備|太陽光発電施設|太陽光発電事業|" \
               r"風力発電設備|風力発電施設|風力発電事業|" \
               r"発電設備|発電施設|発電事業|" \
               r"再生可能エネルギー発電設備|再生可能エネルギー発電施設|再生可能エネルギー発電事業|" \
               r"設置事業|設備設置|施設設置|" \
               r"事業区域|事業地|事業者|事業主|事業計画|" \
               r"住民説明|住民等説明会|周辺住民|近隣住民|関係住民|周辺関係者|近隣関係者|利害関係者|" \
               r"土地所有者|所有者|占有者|管理者|" \
               r"調整池|貯水池|遊砂池|" \
               r"擁壁|法面|のり面|" \
               r"パワーコンディショナー|太陽電池モジュール|架台|" \
               r"低周波音|騒音|振動|反射光|" \
               r"防災|減災|" \
               r"造成|掘削|盛土|切土|伐採|" \
               r"排水|雨水|流出|" \
               r"営農型太陽光発電|ソーラーシェアリング|" \
               r"発電出力|出力|容量)"
    
    # パターンマッチング
    pattern = re.compile(rf"[一-龥ぁ-んァ-ンA-Za-z0-9・ー（）\(\)／/\-]+(?:{suffixes})")
    cands = set(re.findall(pattern, text))
    
    # 法律名を抽出
    law_pattern = re.compile(r"[一-龥ぁ-んァ-ンA-Za-z0-9・ー（）\(\)]+法")
    cands |= set(re.findall(law_pattern, text))
    
    # 条例名・規則名を抽出
    ordinance_pattern = re.compile(r"[一-龥ぁ-んァ-ンA-Za-z0-9・ー（）\(\)]+条例")
    cands |= set(re.findall(ordinance_pattern, text))
    
    # 清掃・フィルタリング
    cleaned = set()
    for w in cands:
        w = w.strip("（）() \n\t「」『』【】、。")
        if 2 <= len(w) <= 50:  # 長さ制限を拡大
            # 純粋な数字のみを除外
            if not re.fullmatch(r"[0-9０-９\.\-／/]+", w):
                cleaned.add(w)
    
    return sorted(cleaned)

print("用語抽出中...")
cands = extract_candidates(texts)

print(f"\n抽出された候補用語数: {len(cands):,}")
print(f"\n最初の50個:\n{cands[:50]}")

用語抽出中...

抽出された候補用語数: 872

最初の50個:
['1)新得町立地適正化計画(都市再生特別措置法', '25年法', '4年法', 'あっせん', 'あらかじめ再生可能エネルギー電気の利用の促進に関する特別措置法', 'あらかじめ開催の日時及び場所を地域住民等の見やすい場所において行う掲示その他の適切な方法', 'かつらぎ町太陽光発電設備の設置に関する条例', 'から60日を経過する日までの間に再生可能エネルギー発電事業に着手しようとする事業者に係るこの条例', 'が暴力団員による不当な行為の防止等に関する法律(平成3年法', 'この条例', 'この条例の施行に際し現に法', 'この条例の施行の日前に工事に着手している太陽光発電設備設置事業及びこの条例', 'この条例の施行の日前に法', 'この条例の規定により公表する書類等に個人情報(個人情報の保護に関する法律(平成15年法', 'この条例及びこの条例', 'この条例及び関係法', 'する事業(建築基準法(昭和25年法', 'その他の法', 'その他の関係法', 'その他市長が適当と認める方法', 'その他関係する法', 'その他関係法', 'その全部を建築基準法(昭和25年法', 'その取消しの日から5年を経過しない者(当該許可を取り消された者が法', 'その取消しの日から起算して5年を経過しないもの(当該発電事業許可を取り消された者が法', 'その方法', 'その旨並びに当該命令を受けた者の氏名及び住所(法', 'その旨及び勧告に従わない者の氏名及び住所(法', 'その法', 'その法定代理人の住民票の写し(法定代理人が法', 'その者の氏名及び住所(法', 'その設置に関して盛土規制法', 'その際処分の方法', 'な設置等に関する条例', 'に法', 'に規定する説明会の開催以外の方法', 'の他の市長が定める方法', 'の区域(再生可能エネルギー電気の利用の促進に関する特別措置法', 'の区域及び第1種農地(同法', 'の実施に際し関係法', 'の擁壁の構造は盛土規制法', 'の敷地及び同条例', 'の敷地及び同法', 'の氏名及び住所(法', 'びに法', 'パワーコンディショナー', '上記の条例', '並びに条例', '主たる事務所の所在地)氏名(法', '事前協議が終了した後

In [3]:
# 拡張されたキュレーションリストと法律リスト

# 重要な行政・法律用語のキュレーションリスト
curated = {
    # 手続き関連
    "事前協議","事後協議","協議","説明会","住民説明","住民等説明会","周辺住民説明会",
    "届出","申請","許可","認可","承認","同意","通知","報告",
    
    # 計画・事業関連
    "事業計画","事業区域","事業地","事業者","事業主",
    "発電事業","設置事業","再生可能エネルギー発電事業",
    "設置許可","設置事業許可申請書","発電設備設置許可申請書",
    "事前協議終了通知書","工事完了届","廃止届",
    
    # 関係者
    "近隣区域","近隣住民","近隣住民等","周辺住民","周辺関係者",
    "関係住民","関係住民等","利害関係者","土地所有者","土地所有者等",
    "占有者","管理者",
    
    # 文書類
    "標識","看板","意見書","協議実施報告書","説明会実施報告書",
    "資金計画","資金計画書","工程表","工事工程表",
    "構造図","位置図","配置図","現況平面図","断面図","立面図",
    "造成計画平面図","排水計画平面図","公図","登記事項証明書",
    "現況写真","完成写真","住民票の写し",
    
    # 施設・設備関連
    "太陽光発電設備","太陽光発電施設","風力発電設備","風力発電施設",
    "再生可能エネルギー発電設備","再生可能エネルギー発電施設",
    "太陽電池モジュール","パワーコンディショナー","架台","送電線",
    "営農型太陽光発電設備","ソーラーシェアリング",
    "発電出力","出力","容量",
    
    # 土木・工事関連
    "撤去","解体","除去","廃止","廃棄","処分",
    "原状回復","復旧","維持管理","保守点検","点検",
    "擁壁","法面","のり面","調整池","貯水池","遊砂池",
    "造成","掘削","盛土","切土","伐採","整地",
    "排水","雨水","流出","防災","減災",
    
    # 環境影響
    "低反射","低周波音","騒音","振動","反射光","景観",
    "自然環境","生活環境","周辺環境","住環境",
    
    # 区域指定
    "禁止区域","抑制区域","規制区域","誘導区域",
    "保全区域","特別保全区域","保護区域","特別保護地区",
    "市街化区域","市街化調整区域","用途地域",
    
    # 自然・保安
    "保安林","国有林","民有林","農用地","農用地区域","農地",
    "土砂災害特別警戒区域","土砂災害警戒区域",
    "急傾斜地崩壊危険区域","地すべり防止区域",
    "砂防指定地","砂防設備",
    "河川区域","河川保全区域","河川予定地",
    "海岸区域","海岸保全区域","港湾区域",
    
    # 公園・景観
    "国立公園","国定公園","県立自然公園","自然公園",
    "特別地域","普通地域",
    "景観計画区域","景観地区","風致地区",
    
    # 文化財・生態系
    "文化財","史跡","名勝","天然記念物","埋蔵文化財",
    "伝統的建造物群保存地区",
    "鳥獣保護区","特別保護地区","狩猟区",
    "特定保全種","希少野生動植物",
}

# 主要な関連法律のリスト
laws = {
    # エネルギー関連
    "再生可能エネルギー電気の利用の促進に関する特別措置法",
    "再エネ特措法","FIT法",
    "電気事業法",
    
    # 防災・砂防関連
    "砂防法","森林法","治山治水法",
    "地すべり等防止法",
    "急傾斜地の崩壊による災害の防止に関する法律",
    "急傾斜地法",
    "土砂災害警戒区域等における土砂災害防止対策の推進に関する法律",
    "土砂災害防止法",
    
    # 河川・海岸
    "河川法","海岸法","港湾法","漁港漁場整備法",
    
    # 農地・農業
    "農地法",
    "農業振興地域の整備に関する法律",
    "農振法",
    
    # 自然環境
    "自然公園法","自然環境保全法",
    "鳥獣の保護及び管理並びに狩猟の適正化に関する法律",
    "鳥獣保護管理法",
    "絶滅のおそれのある野生動植物の種の保存に関する法律",
    "種の保存法",
    
    # 文化財
    "文化財保護法",
    
    # 都市計画・景観
    "都市計画法","景観法","建築基準法",
    
    # 環境一般
    "環境基本法","環境影響評価法",
}

# マージと基本フィルタリング
filtered = set()
generic = {"区域","地域","地区","協議","同意","許可","基準","規則","手続",
           "申請書","報告書","通知書","証明書","意見","説明会","届出","届"}

for w in cands:
    if len(w) <= 40 and w not in generic:
        w = w.replace(" )", ")").replace("( ", "(").strip()
        # 明らかに壊れた法律名の断片を除外
        if "法(" in w and "法律第" not in w and ")" not in w:
            continue
        if w.startswith("第") and ("条" in w or "項" in w or "号" in w) and len(w) <= 5:
            continue  # 「第1条」のような条項番号を除外
        filtered.add(w)

# キュレーションと法律リストをマージ
terms = sorted(curated | laws | filtered)

print(f"フィルタリング後の用語数: {len(terms):,}")
print(f"\n最初の100個:\n{terms[:100]}")

フィルタリング後の用語数: 869

最初の100個:
['1)新得町立地適正化計画(都市再生特別措置法', '25年法', '4年法', 'FIT法', 'あっせん', 'あらかじめ再生可能エネルギー電気の利用の促進に関する特別措置法', 'かつらぎ町太陽光発電設備の設置に関する条例', 'が暴力団員による不当な行為の防止等に関する法律(平成3年法', 'この条例', 'この条例の施行に際し現に法', 'この条例の施行の日前に工事に着手している太陽光発電設備設置事業及びこの条例', 'この条例の施行の日前に法', 'この条例及びこの条例', 'この条例及び関係法', 'その他の法', 'その他の関係法', 'その他市長が適当と認める方法', 'その他関係する法', 'その他関係法', 'その取消しの日から5年を経過しない者(当該許可を取り消された者が法', 'その方法', 'その旨並びに当該命令を受けた者の氏名及び住所(法', 'その旨及び勧告に従わない者の氏名及び住所(法', 'その法', 'その法定代理人の住民票の写し(法定代理人が法', 'その者の氏名及び住所(法', 'その設置に関して盛土規制法', 'その際処分の方法', 'な設置等に関する条例', 'に法', 'に規定する説明会の開催以外の方法', 'のり面', 'の他の市長が定める方法', 'の区域(再生可能エネルギー電気の利用の促進に関する特別措置法', 'の区域及び第1種農地(同法', 'の実施に際し関係法', 'の擁壁の構造は盛土規制法', 'の敷地及び同条例', 'の敷地及び同法', 'の氏名及び住所(法', 'びに法', 'ソーラーシェアリング', 'パワーコンディショナー', '上記の条例', '並びに条例', '主たる事務所の所在地)氏名(法', '事前協議', '事前協議が終了した後の条例', '事前協議又は変更事前協議において指示された事項等への対応及び関係法', '事前協議終了通知書', '事後協議', '事故等報告書士別市の自然環境等と再生可能エネルギー発電事業との調和に関する条例', '事業に関する法', '事業の実施に係る書面の提出方法', '事業の計画又はその概要を記載した印刷物の配布その他適切な方法', '事業の計画及び実施並びに関係法', '事業主', '事業主及び工事

In [4]:
# MeCab用ユーザー辞書CSV生成関数

def to_csv_rows(terms, default_cost=8000):
    """
    MeCab IPA辞書形式のCSVレコードを生成
    形式: 表層形,left_id,right_id,cost,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
    """
    rows = []
    for t in terms:
        row = [
            t,           # 表層形
            "",          # left_id (空欄)
            "",          # right_id (空欄)
            str(default_cost),  # cost (低いほど優先度高い、8000は安全側)
            "名詞",      # 品詞
            "固有名詞",  # 品詞細分類1
            "一般",      # 品詞細分類2
            "*",         # 品詞細分類3
            "*",         # 活用型
            "*",         # 活用形
            t,           # 原形(表層形と同じ)
            "*",         # 読み(未設定)
            "*"          # 発音(未設定)
        ]
        rows.append(row)
    return rows

# CSV生成
rows = to_csv_rows(terms, default_cost=8000)

# 確認用にDataFrame化
header = ["surface","left_id","right_id","cost","pos","pos1","pos2","pos3","conj","infl","base","reading","pron"]
df_dict = pd.DataFrame(rows, columns=header)

print(f"生成されたレコード数: {len(rows):,}")
print(f"\n最初の20行:\n{df_dict.head(20)}")

# 一時保存
temp_output = "/home/ubuntu/cur/isep/solar_ordinance_userdic_v1_temp.csv"
df_dict.to_csv(temp_output, index=False, header=False, encoding="utf-8")
print(f"\n一時ファイル出力: {temp_output}")

生成されたレコード数: 869

最初の20行:
                                  surface left_id right_id  cost pos  pos1  \
0                  1)新得町立地適正化計画(都市再生特別措置法                   8000  名詞  固有名詞   
1                                    25年法                   8000  名詞  固有名詞   
2                                     4年法                   8000  名詞  固有名詞   
3                                    FIT法                   8000  名詞  固有名詞   
4                                    あっせん                   8000  名詞  固有名詞   
5         あらかじめ再生可能エネルギー電気の利用の促進に関する特別措置法                   8000  名詞  固有名詞   
6                   かつらぎ町太陽光発電設備の設置に関する条例                   8000  名詞  固有名詞   
7           が暴力団員による不当な行為の防止等に関する法律(平成3年法                   8000  名詞  固有名詞   
8                                    この条例                   8000  名詞  固有名詞   
9                           この条例の施行に際し現に法                   8000  名詞  固有名詞   
10  この条例の施行の日前に工事に着手している太陽光発電設備設置事業及びこの条例                   8000  名詞  固有名詞   
11                           この条例の施行の日前

In [5]:
# 詳細フィルタリング: ノイズとなる用語を除去

# 除外すべき悪いパターン
bad_patterns = [
    re.compile(r'^\d+\)'),          # 数字で始まる番号
    re.compile(r'^第\d+条'),        # 条項番号
    re.compile(r'^第\d+項'),        
    re.compile(r'^第\d+号'),
    re.compile(r'法\(昭和'),        # 不完全な法律名
    re.compile(r'法\(平成'),
    re.compile(r'法\(令和'),
    re.compile(r'\(昭和\d+年法律第\d+号$'),  # 法律番号のみ
    re.compile(r'\(平成\d+年法律第\d+号$'),
    re.compile(r'\(令和\d+年法律第\d+号$'),
    re.compile(r'^[0-9]+年[0-9]+月[0-9]+日$'),  # 日付
    re.compile(r'の写し$'),         # 「〜の写し」
    re.compile(r'等の写し$'),
]

# 除外すべき完全一致の用語
bad_exact = {
    "その法","同法","当該法","本法",
    "この条例","当該条例","本条例",
    "氏名及び住所","住民票の写し","登記事項証明書の写し",
    "の規定","同条","同項","同号","前条","前項","前号","次条","次項",
    "以下","以上","以内","未満",
}

# 除外すべきキーワードを含む用語
bad_keywords = ["の規定", "同条", "同項", "同号", "前各号", "次に掲げる", "写し"]

refined = set()
for w in terms:
    # 悪いパターンに該当するかチェック
    if any(p.search(w) for p in bad_patterns):
        continue
    # 完全一致の除外リストに該当するかチェック
    if w in bad_exact:
        continue
    # 除外キーワードを含むかチェック
    if any(kw in w for kw in bad_keywords):
        continue
    # 過度に短い用語(1文字)を除外
    if len(w) < 2:
        continue
    
    refined.add(w)

print(f"詳細フィルタリング後: {len(refined):,} 用語")
print(f"\n最初の100個:\n{sorted(refined)[:100]}")

詳細フィルタリング後: 809 用語

最初の100個:
['25年法', '4年法', 'FIT法', 'あっせん', 'あらかじめ再生可能エネルギー電気の利用の促進に関する特別措置法', 'かつらぎ町太陽光発電設備の設置に関する条例', 'が暴力団員による不当な行為の防止等に関する法律(平成3年法', 'この条例の施行に際し現に法', 'この条例の施行の日前に工事に着手している太陽光発電設備設置事業及びこの条例', 'この条例の施行の日前に法', 'この条例及びこの条例', 'この条例及び関係法', 'その他の法', 'その他の関係法', 'その他市長が適当と認める方法', 'その他関係する法', 'その他関係法', 'その取消しの日から5年を経過しない者(当該許可を取り消された者が法', 'その方法', 'その旨並びに当該命令を受けた者の氏名及び住所(法', 'その旨及び勧告に従わない者の氏名及び住所(法', 'その者の氏名及び住所(法', 'その設置に関して盛土規制法', 'その際処分の方法', 'な設置等に関する条例', 'に法', 'に規定する説明会の開催以外の方法', 'のり面', 'の他の市長が定める方法', 'の区域(再生可能エネルギー電気の利用の促進に関する特別措置法', 'の区域及び第1種農地(同法', 'の実施に際し関係法', 'の擁壁の構造は盛土規制法', 'の敷地及び同法', 'の氏名及び住所(法', 'びに法', 'ソーラーシェアリング', 'パワーコンディショナー', '上記の条例', '並びに条例', '主たる事務所の所在地)氏名(法', '事前協議', '事前協議が終了した後の条例', '事前協議又は変更事前協議において指示された事項等への対応及び関係法', '事前協議終了通知書', '事後協議', '事故等報告書士別市の自然環境等と再生可能エネルギー発電事業との調和に関する条例', '事業に関する法', '事業の実施に係る書面の提出方法', '事業の計画又はその概要を記載した印刷物の配布その他適切な方法', '事業の計画及び実施並びに関係法', '事業主', '事業主及び工事施工者の氏名及び住所(法', '事業区域', '事業区域から国道又は道路法', '事業区域が存する行政区等(皆野町行政区設置条例(平成19年皆野町条例', 

In [6]:
# 最終精査: 法律名の厳格なチェック

# ホワイトリスト化された法律名(完全な正式名称)
law_whitelist = laws | {
    "再エネ特措法", "FIT法", "急傾斜地法", "農振法",
    "鳥獣保護管理法", "種の保存法", "土砂災害防止法",
}

final_terms = set()
for w in refined:
    # 「法」を含む用語の厳格なチェック
    if "法" in w:
        # ホワイトリストにあれば許可
        if w in law_whitelist:
            final_terms.add(w)
            continue
        # 「〜法律」「〜法令」なども除外
        if "法律" in w or "法令" in w or "法(" in w:
            continue
        # 「〜に関する法律」など正式な法律名パターンでない場合は除外
        if not any(pattern in w for pattern in ["に関する法律", "による法律", "の法律"]):
            # ただし、シンプルな「〜法」(例:森林法、河川法)は許可
            if w.endswith("法") and len(w) <= 15 and w not in law_whitelist:
                # 短い法律名らしきものはホワイトリストにあれば許可、なければスキップ
                continue
    
    # 「条例」を含む場合も同様のチェック
    if "条例" in w:
        # あまりに長い条例名や不完全な条例名は除外
        if len(w) > 35 or "(" in w:
            continue
    
    final_terms.add(w)

# キュレーションリストとホワイトリストは必ず含める
final_terms |= curated | law_whitelist

print(f"最終用語数: {len(final_terms):,}")
print(f"\n最終用語リスト(最初の150個):\n{sorted(final_terms)[:150]}")

最終用語数: 466

最終用語リスト(最初の150個):
['FIT法', 'あっせん', 'あらかじめ再生可能エネルギー電気の利用の促進に関する特別措置法', 'かつらぎ町太陽光発電設備の設置に関する条例', 'この条例及びこの条例', 'その取消しの日から5年を経過しない者(当該許可を取り消された者が法', 'その旨並びに当該命令を受けた者の氏名及び住所(法', 'その旨及び勧告に従わない者の氏名及び住所(法', 'な設置等に関する条例', 'に規定する説明会の開催以外の方法', 'のり面', 'の区域(再生可能エネルギー電気の利用の促進に関する特別措置法', 'ソーラーシェアリング', 'パワーコンディショナー', '上記の条例', '並びに条例', '事前協議', '事前協議が終了した後の条例', '事前協議又は変更事前協議において指示された事項等への対応及び関係法', '事前協議終了通知書', '事後協議', '事業の計画又はその概要を記載した印刷物の配布その他適切な方法', '事業主', '事業主及び工事施工者の氏名及び住所(法', '事業区域', '事業地', '事業地域内に居住する者及び所在する法', '事業終了後の撤去及び廃棄物の処理に充てる費用を計画的な積立て等の方法', '事業者', '事業者が本条例', '事業者が条例', '事業者に電電話番号以外の連絡方法', '事業者の氏名及び住所(事業者が法', '事業計画', '事業計画が条例', '事業計画と相違しているおそれがある場合又は条例', '五條市太陽光発電設備の適正な設置及び管理に関する条例', '京田辺市太陽光発電設備の適正な設置及び維持管理に関する条例', '京田辺市条例', '今金町自然環境等と再生可能エネルギー発電事業との調和に関する条例', '他自治体の条例', '令和6年3月15日条例', '令和6年3月五條市条例', '令和6年6月4日条例', '令和6年9月26日条例', '令和6年条例', '令和6条例', '令和7年3月14日条例', '令和7年条例', '令和7年松江市条例', '伊達市自然環境等と再生可能エネルギー発電事業との調和に関する条例', '伐採', '伐採した木竹及び除去した木竹の根等は関係法', '伝統的建造物群保存地区', '位置図', '低反射'

In [7]:
# 最終ユーザー辞書CSV出力

# 最終的なCSVレコードを生成
final_rows = to_csv_rows(sorted(final_terms), default_cost=8000)

# 出力ファイルパス
output_path = "/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2.csv"

# CSV書き込み
with open(output_path, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for r in final_rows:
        writer.writerow(r)

print(f"✓ ユーザー辞書CSVファイルを生成しました")
print(f"  出力先: {output_path}")
print(f"  用語数: {len(final_rows):,}")
print(f"  処理した条文データ: {len(df):,} 行")
print(f"  元テキスト長: {len(texts):,} 文字")

# 統計情報
print(f"\n=== 統計情報 ===")
term_lengths = [len(t) for t in final_terms]
print(f"  平均用語長: {sum(term_lengths)/len(term_lengths):.1f} 文字")
print(f"  最短用語長: {min(term_lengths)} 文字")
print(f"  最長用語長: {max(term_lengths)} 文字")

# サンプル表示
print(f"\n=== 用語サンプル(ランダム50個) ===")
import random
sample_terms = random.sample(sorted(final_terms), min(50, len(final_terms)))
for i, term in enumerate(sample_terms, 1):
    print(f"{i:2d}. {term}")

output_path

✓ ユーザー辞書CSVファイルを生成しました
  出力先: /home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2.csv
  用語数: 466
  処理した条文データ: 102 行
  元テキスト長: 523,040 文字

=== 統計情報 ===
  平均用語長: 12.7 文字
  最短用語長: 2 文字
  最長用語長: 40 文字

=== 用語サンプル(ランダム50個) ===
 1. 村公式ウェブサイトへの掲載その他の方法
 2. 合併又は分割により設立されている法
 3. 禁止区域
 4. 遊砂池
 5. 住環境
 6. 公告
 7. 農用地区域(再生可能エネルギー電気の利用の促進に関する特別措置法
 8. 送電線
 9. 架台
10. 特定事業の実施に当たって要する他の法
11. 住みよい環境づくり条例及び関係法
12. 再生可能エネルギー発電事業計画及び再生可能エネルギー発電事業に係る関係法
13. 令和6年3月五條市条例
14. 撤去
15. 騒音
16. 大町市太陽光発電設備の設置管理等に関する条例
17. 港湾区域
18. 角田市自然環境等と再生可能エネルギー発電事業との調和に関する条例
19. パワーコンディショナー
20. 廃止届
21. 用途地域
22. 皆野町太陽光発電設備の適正な設置等に関する条例
23. 愛知県立自然公園条例
24. 太陽光発電施設等設計するにあたり各関係法
25. 河川区域
26. 用その他の情報通信の技術を利用する方法
27. 本庄市自然環境等と再生可能エネルギー発電事業との調和に関する条例
28. 太陽光発電設備の維持管理を行う者の氏名(法
29. 文化財
30. 備の適正な設置等に関する条例
31. 条例第13条並びに条例
32. 南砺市太陽光発電設備の設置の規制等に関する条例
33. 町の施策及びこの条例
34. 急傾斜地の崩壊による災害の防止に関する法律
35. 村の施策及びこの条例
36. 地域住民に対して設置事業に係る計画の内容及び維持管理等の方法
37. 五條市太陽光発電設備の適正な設置及び管理に関する条例
38. 点検
39. 再生可能エネルギー発電施設
40. 桜井市太陽光発電設備の適正な設置及び管理に関する条例
41. 豊橋

'/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2.csv'

In [8]:
# さらなる精査: 長すぎる用語や明らかに不適切な用語を除外

# 不要なパターンをさらに強化
additional_bad_patterns = [
    re.compile(r'\(.*\).*\(.*\)'),  # 複数の括弧を含む
    re.compile(r'.*法\(.*'),         # 「法(」を含む不完全な法律名
    re.compile(r'.*条例\(.*'),       # 「条例(」を含む不完全な条例名
    re.compile(r'^その'),            # 「その」で始まる
    re.compile(r'^この'),            # 「この」で始まる(一部例外を除く)
    re.compile(r'^当該'),            # 「当該」で始まる
    re.compile(r'^上記'),            # 「上記」で始まる
    re.compile(r'氏名及び住所'),     # 定型フレーズ
    re.compile(r'を記載した.*方法'),  # 定型フレーズ
]

# さらに精査
very_final_terms = set()
for w in final_terms:
    # 追加の悪いパターンチェック
    if any(p.search(w) for p in additional_bad_patterns):
        # ただし、重要なキュレーション用語は保護
        if w not in curated and w not in law_whitelist:
            continue
    
    # 長すぎる用語を除外(30文字以上)
    if len(w) > 30:
        if w not in curated and w not in law_whitelist:
            continue
    
    # 明らかに文章の一部と思われるもの(「及び」「又は」「並びに」を含む)
    if any(x in w for x in ['及び', '又は', '並びに', 'その他']):
        # ただし、正式な法律名などは除く
        if not any(x in w for x in ['に関する法律', '保護及び管理', '設置及び', '設置又は']):
            if w not in curated and w not in law_whitelist:
                if len(w) > 15:  # 15文字以上の複雑な構文は除外
                    continue
    
    very_final_terms.add(w)

print(f"最終精査後の用語数: {len(very_final_terms):,}")
print(f"\n削除された用語数: {len(final_terms) - len(very_final_terms):,}")

最終精査後の用語数: 368

削除された用語数: 98


In [9]:
# 最終版のユーザー辞書CSV出力

# 最終的なCSVレコードを生成
very_final_rows = to_csv_rows(sorted(very_final_terms), default_cost=8000)

# 出力ファイルパス
final_output_path = "/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv"

# CSV書き込み
with open(final_output_path, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for r in very_final_rows:
        writer.writerow(r)

print(f"=" * 70)
print(f"✓ MeCab用ユーザー辞書CSVファイルを生成しました")
print(f"=" * 70)
print(f"出力先: {final_output_path}")
print(f"登録用語数: {len(very_final_rows):,}")
print(f"処理した条文データ: {len(df):,} 行({len(df[df['区分']=='条例'])} 条例, {len(df[df['区分']=='施行規則'])} 施行規則)")
print(f"元テキスト総文字数: {len(texts):,} 文字")
print(f"=" * 70)

# 統計情報
term_lengths = [len(t) for t in very_final_terms]
print(f"\n【統計情報】")
print(f"  平均用語長: {sum(term_lengths)/len(term_lengths):.1f} 文字")
print(f"  最短用語長: {min(term_lengths)} 文字")
print(f"  最長用語長: {max(term_lengths)} 文字")

# カテゴリ別集計
law_count = sum(1 for t in very_final_terms if '法' in t and t in law_whitelist)
ordinance_count = sum(1 for t in very_final_terms if '条例' in t)
area_count = sum(1 for t in very_final_terms if any(x in t for x in ['区域', '地域', '地区']))
equipment_count = sum(1 for t in very_final_terms if any(x in t for x in ['設備', '施設', 'パワーコンディショナー', 'モジュール']))

print(f"\n【カテゴリ別集計】")
print(f"  法律名: {law_count}")
print(f"  条例名: {ordinance_count}")
print(f"  区域・地域関連: {area_count}")
print(f"  設備・施設関連: {equipment_count}")

# サンプル表示
print(f"\n【用語サンプル(ランダム50個)】")
import random
sample_terms = random.sample(sorted(very_final_terms), min(50, len(very_final_terms)))
for i, term in enumerate(sorted(sample_terms), 1):
    print(f"  {i:2d}. {term}")

print(f"\n{'=' * 70}")
print(f"処理完了!")
print(f"{'=' * 70}")

final_output_path

✓ MeCab用ユーザー辞書CSVファイルを生成しました
出力先: /home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv
登録用語数: 368
処理した条文データ: 102 行(53 条例, 49 施行規則)
元テキスト総文字数: 523,040 文字

【統計情報】
  平均用語長: 9.3 文字
  最短用語長: 2 文字
  最長用語長: 30 文字

【カテゴリ別集計】
  法律名: 31
  条例名: 109
  区域・地域関連: 46
  設備・施設関連: 61

【用語サンプル(ランダム50個)】
   1. 事業者
   2. 事業者が条例
   3. 令和6年9月26日条例
   4. 住民説明
   5. 保全
   6. 保守
   7. 公図
   8. 公表
   9. 再生可能エネルギー発電事業
  10. 出力
  11. 協議実施報告書
  12. 南箕輪村太陽光発電施設の設置等に関する条例
  13. 土地所有者
  14. 地元団体等からの意見聴取等の方法
  15. 報告
  16. 大町市条例
  17. 太陽光発電施設の維持管理を行う者の氏名(法
  18. 太陽光発電施設を廃止した後の措置の方法
  19. 太陽光発電施設等設計するにあたり各関係法
  20. 太陽電池モジュール
  21. 市の施策及びこの条例
  22. 廃棄
  23. 建築基準法
  24. 文化財
  25. 景観計画区域
  26. 未成年者の法定代理人(法定代理人が法
  27. 条例第11条に規定する条例
  28. 条例第12条第2項の事業計画に条例
  29. 条例第12条第3項及び条例
  30. 条例第13条並びに条例
  31. 条例第14条第2項において準用する条例
  32. 河川予定地
  33. 河川法
  34. 減災
  35. 特別地域
  36. 環境基本法
  37. 用途地域
  38. 禁止区域
  39. 福島市条例
  40. 種の保存法
  41. 自然環境
  42. 規制区域
  43. 設置事業
  44. 設置等に関する条例
  45. 許可事業者及びこの条例
  46. 許可申請者が条例
  47. 農用地


'/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv'

In [10]:
# さらなるクリーニング: 明らかに文章の断片である用語を除外

# より厳格なフィルタリング
ultra_final_terms = set()

# 除外すべき開始パターン
bad_starts = ['な設置', 'に規定', 'の区域', '並びに', '事業者が', '事業計画が', '他自治体の']

# 除外すべき終了パターン  
bad_ends = ['の条例', '以外の方法', 'の連絡方法', 'した後の条例']

# 除外すべき日付パターン
date_pattern = re.compile(r'令和\d+年.*条例')

for w in very_final_terms:
    # 悪い開始パターン
    if any(w.startswith(x) for x in bad_starts):
        if w not in curated and w not in law_whitelist:
            continue
    
    # 悪い終了パターン
    if any(w.endswith(x) for x in bad_ends):
        if w not in curated and w not in law_whitelist:
            continue
    
    # 日付を含む条例名は除外(ただし完全な条例名は保持)
    if date_pattern.match(w) and len(w) < 20:
        continue
    
    # 1文字の用語を除外
    if len(w) == 1:
        continue
    
    # 2文字で一般的すぎる用語を除外
    if len(w) == 2 and w in ['保全', '保存', '保守', '保護', '防止', '防災', '除去', '撤去', '伐採', '造成', '掘削', '盛土', '切土', '排水', '雨水', '出力', '容量', '設置', '管理', '維持', '点検', '騒音', '振動', '景観', '協議', '同意', '許可', '認可', '承認', '通知', '報告', '申請', '届出']:
        if w not in curated:
            continue
    
    ultra_final_terms.add(w)

print(f"最終クリーニング後の用語数: {len(ultra_final_terms):,}")
print(f"削除された用語数: {len(very_final_terms) - len(ultra_final_terms):,}")

# 削除された用語を確認
removed = sorted(very_final_terms - ultra_final_terms)
if removed:
    print(f"\n削除された用語の例(最初の30個):")
    for i, t in enumerate(removed[:30], 1):
        print(f"  {i:2d}. {t}")

最終クリーニング後の用語数: 330
削除された用語数: 38

削除された用語の例(最初の30個):
   1. な設置等に関する条例
   2. に規定する説明会の開催以外の方法
   3. の区域(再生可能エネルギー電気の利用の促進に関する特別措置法
   4. 並びに条例
   5. 事前協議が終了した後の条例
   6. 事業者が本条例
   7. 事業者が条例
   8. 事業者に電電話番号以外の連絡方法
   9. 事業計画が条例
  10. 他自治体の条例
  11. 令和6年3月15日条例
  12. 令和6年3月五條市条例
  13. 令和6年6月4日条例
  14. 令和6年9月26日条例
  15. 令和6年条例
  16. 令和7年3月14日条例
  17. 令和7年条例
  18. 令和7年松江市条例
  19. 保全
  20. 保存
  21. 保守
  22. 保護
  23. 地方公共団体が行う太陽光発電設備設置事業はこの条例
  24. 市が実施する施策及びこの条例
  25. 市の施策及びこの条例
  26. 市長がこの条例
  27. 村の施策及びこの条例
  28. 次の表の左欄に掲げるこの条例
  29. 災害の防止その他この条例
  30. 町の施策及びこの条例


In [11]:
# 最終版ユーザー辞書CSVの出力

# 最終的なCSVレコードを生成
ultra_final_rows = to_csv_rows(sorted(ultra_final_terms), default_cost=8000)

# 出力ファイルパス(最終版)
final_output_path = "/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv"

# CSV書き込み
with open(final_output_path, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for r in ultra_final_rows:
        writer.writerow(r)

print(f"\n{'=' * 75}")
print(f"  ✓ MeCab用ユーザー辞書CSVファイルの生成が完了しました")
print(f"{'=' * 75}")
print(f"\n【出力情報】")
print(f"  ファイル名: {final_output_path}")
print(f"  登録用語数: {len(ultra_final_rows):,} 語")
print(f"\n【入力データ】")
print(f"  処理した条文数: {len(df):,} 件")
print(f"    └ 条例: {len(df[df['区分']=='条例'])} 件")
print(f"    └ 施行規則: {len(df[df['区分']=='施行規則'])} 件")
print(f"  総文字数: {len(texts):,} 文字")
print(f"\n【処理統計】")
print(f"  初期抽出候補: {len(cands):,} 語")
print(f"  フィルタリング後: {len(filtered):,} 語")
print(f"  詳細精査後: {len(refined):,} 語")
print(f"  最終精査後: {len(final_terms):,} 語")
print(f"  クリーニング後: {len(very_final_terms):,} 語")
print(f"  最終版: {len(ultra_final_terms):,} 語")

# 統計情報
term_lengths = [len(t) for t in ultra_final_terms]
print(f"\n【用語長統計】")
print(f"  平均: {sum(term_lengths)/len(term_lengths):.1f} 文字")
print(f"  最短: {min(term_lengths)} 文字")
print(f"  最長: {max(term_lengths)} 文字")
print(f"  中央値: {sorted(term_lengths)[len(term_lengths)//2]} 文字")

# カテゴリ別集計
law_terms = [t for t in ultra_final_terms if '法' in t]
ordinance_terms = [t for t in ultra_final_terms if '条例' in t]
area_terms = [t for t in ultra_final_terms if any(x in t for x in ['区域', '地域', '地区'])]
equipment_terms = [t for t in ultra_final_terms if any(x in t for x in ['設備', '施設'])]
procedure_terms = [t for t in ultra_final_terms if any(x in t for x in ['協議', '説明会', '届出', '申請', '許可'])]

print(f"\n【カテゴリ別用語数】")
print(f"  法律関連: {len(law_terms)} 語")
print(f"  条例関連: {len(ordinance_terms)} 語")
print(f"  区域・地域: {len(area_terms)} 語")
print(f"  設備・施設: {len(equipment_terms)} 語")
print(f"  手続き関連: {len(procedure_terms)} 語")

# サンプル表示
print(f"\n【用語サンプル(ランダム50語)】")
sample_terms = random.sample(sorted(ultra_final_terms), min(50, len(ultra_final_terms)))
for i, term in enumerate(sorted(sample_terms), 1):
    if i % 2 == 1:
        print(f"  {i:2d}. {term:<35}", end="")
    else:
        print(f"  {i:2d}. {term}")
if len(sample_terms) % 2 == 1:
    print()  # 改行

print(f"\n{'=' * 75}")
print(f"  処理が正常に完了しました!")
print(f"{'=' * 75}\n")

final_output_path


  ✓ MeCab用ユーザー辞書CSVファイルの生成が完了しました

【出力情報】
  ファイル名: /home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv
  登録用語数: 330 語

【入力データ】
  処理した条文数: 102 件
    └ 条例: 53 件
    └ 施行規則: 49 件
  総文字数: 523,040 文字

【処理統計】
  初期抽出候補: 872 語
  フィルタリング後: 745 語
  詳細精査後: 809 語
  最終精査後: 466 語
  クリーニング後: 368 語
  最終版: 330 語

【用語長統計】
  平均: 9.2 文字
  最短: 2 文字
  最長: 30 文字
  中央値: 5 文字

【カテゴリ別用語数】
  法律関連: 61 語
  条例関連: 82 語
  区域・地域: 45 語
  設備・施設: 58 語
  手続き関連: 19 語

【用語サンプル(ランダム50語)】
   1. のり面                                   2. 事後協議
   3. 佐久穂町太陽光発電施設の設置及び維持管理に関する条例            4. 保護区域
   5. 史跡                                    6. 周辺環境
   7. 営農型太陽光発電設備                            8. 土地所有者等
   9. 土砂災害警戒区域等における土砂災害防止対策の推進に関する法律       10. 大町市太陽光発電設備の設置管理等に関する条例
  11. 完成写真                                 12. 市街化区域
  13. 抑制区域                                 14. 撤去
  15. 新型インフルエンザ等対策特別措置法                    16. 日置市地域と再生可能エネルギー発電事業との共生に関する条例
  17. 景観                                   18. 条例第11条に規定する条例
  19. 条

'/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv'

In [12]:
# 品質チェック: 明らかに不適切な用語を手動確認

# 長い用語(20文字以上)で特定のパターンを含むものをリストアップ
long_terms = [t for t in sorted(ultra_final_terms) if len(t) >= 20]
suspicious_terms = []

for t in long_terms:
    # 括弧を含むものは疑わしい
    if '(' in t or '（' in t:
        suspicious_terms.append(t)
    # 「法」や「条例」で終わらない長い文
    elif len(t) >= 25 and not (t.endswith('法') or t.endswith('条例') or t.endswith('法律')):
        suspicious_terms.append(t)

print(f"20文字以上の用語数: {len(long_terms)}")
print(f"疑わしい用語数: {len(suspicious_terms)}")

if suspicious_terms:
    print(f"\n疑わしい用語のリスト:")
    for i, t in enumerate(suspicious_terms, 1):
        print(f"  {i:2d}. [{len(t):2d}文字] {t}")

20文字以上の用語数: 61
疑わしい用語数: 7

疑わしい用語のリスト:
   1. [20文字] 塀(組積造の塀を設置する場合は建築基準法
   2. [20文字] 太陽光発電事業の実施に必要な許認可等(法
   3. [21文字] 太陽光発電施設の維持管理を行う者の氏名(法
   4. [21文字] 太陽光発電設備の維持管理を行う者の氏名(法
   5. [28文字] 景観等と設置事業等との調和が特に必要と認められる区域(法
   6. [30文字] 発電事業終了届(様式第30号)に再生可能エネルギー特別措置法
   7. [20文字] 発電設備の保守点検を行う事業者の氏名(法


In [13]:
# 最終フィルタリング: 疑わしい用語を除外

# 除外する用語のパターン
exclude_patterns = [
    re.compile(r'.*\(.*法$'),  # 「(」を含んで「法」で終わる
    re.compile(r'.*\(.*条例$'),  # 「(」を含んで「条例」で終わる
    re.compile(r'^塀\('),  # 「塀(」で始まる
]

# より厳格に: 「を行う者の氏名」「が特に必要と認められる」などの定型フレーズを含む
exclude_phrases = ['を行う者の氏名', '特に必要と認められる', '様式第', 'に再生可能エネルギー特別措置法']

# 最終版の用語セット
clean_final_terms = set()

for t in ultra_final_terms:
    # 除外パターンチェック
    if any(p.match(t) for p in exclude_patterns):
        continue
    # 除外フレーズチェック
    if any(phrase in t for phrase in exclude_phrases):
        continue
    
    clean_final_terms.add(t)

print(f"最終クリーン後の用語数: {len(clean_final_terms)}")
print(f"除外された用語数: {len(ultra_final_terms) - len(clean_final_terms)}")

# 除外された用語を表示
excluded = sorted(ultra_final_terms - clean_final_terms)
if excluded:
    print(f"\n除外された用語:")
    for i, t in enumerate(excluded, 1):
        print(f"  {i}. {t}")

最終クリーン後の用語数: 319
除外された用語数: 11

除外された用語:
  1. 塀(組積造の塀を設置する場合は建築基準法
  2. 太陽光発電事業の実施に必要な許認可等(法
  3. 太陽光発電施設の維持管理を行う者の氏名(法
  4. 太陽光発電設備の維持管理を行う者の氏名(法
  5. 施行日前に再生可能エネルギー特別措置法
  6. 景観等と設置事業等との調和が特に必要と認められる区域(法
  7. 未成年者の法定代理人(法定代理人が法
  8. 法人の登記事項証明書(事業者が法
  9. 発電事業終了届(様式第30号)に再生可能エネルギー特別措置法
  10. 発電設備の保守点検を行う事業者の氏名(法
  11. 雨水排水処理計画図(排水流域と処理方法


In [14]:
# 最終版ユーザー辞書CSVの出力(クリーン版)

# 最終的なCSVレコードを生成
clean_final_rows = to_csv_rows(sorted(clean_final_terms), default_cost=8000)

# 出力ファイルパス
final_clean_output = "/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv"

# CSV書き込み
with open(final_clean_output, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for r in clean_final_rows:
        writer.writerow(r)

print(f"\n{'=' * 75}")
print(f"  ✓ MeCab用ユーザー辞書CSVファイルの生成が完了しました")
print(f"{'=' * 75}")
print(f"\n【出力情報】")
print(f"  ファイル名: {final_clean_output}")
print(f"  登録用語数: {len(clean_final_rows):,} 語")
print(f"\n【入力データ】")
print(f"  CSVファイル: main2.2.csv")
print(f"  処理した条文数: {len(df):,} 件")
print(f"    ├ 条例: {len(df[df['区分']=='条例'])} 件")
print(f"    └ 施行規則: {len(df[df['区分']=='施行規則'])} 件")
print(f"  総文字数: {len(texts):,} 文字")
print(f"\n【処理統計】")
print(f"  初期抽出候補: {len(cands):,} 語")
print(f"  → フィルタリング後: {len(filtered):,} 語")
print(f"  → 詳細精査後: {len(refined):,} 語")
print(f"  → 最終精査後: {len(final_terms):,} 語")
print(f"  → クリーニング後: {len(very_final_terms):,} 語")
print(f"  → さらに精査後: {len(ultra_final_terms):,} 語")
print(f"  → 最終クリーン版: {len(clean_final_terms):,} 語 ★")

# 統計情報
term_lengths = [len(t) for t in clean_final_terms]
print(f"\n【用語長統計】")
print(f"  平均: {sum(term_lengths)/len(term_lengths):.1f} 文字")
print(f"  最短: {min(term_lengths)} 文字")
print(f"  最長: {max(term_lengths)} 文字")
print(f"  中央値: {sorted(term_lengths)[len(term_lengths)//2]} 文字")

# カテゴリ別集計
法律関連 = len([t for t in clean_final_terms if '法' in t])
条例関連 = len([t for t in clean_final_terms if '条例' in t])
区域地域 = len([t for t in clean_final_terms if any(x in t for x in ['区域', '地域', '地区'])])
設備施設 = len([t for t in clean_final_terms if any(x in t for x in ['設備', '施設'])])
手続き = len([t for t in clean_final_terms if any(x in t for x in ['協議', '説明会', '届出', '申請', '許可', '報告書'])])

print(f"\n【カテゴリ別用語数】")
print(f"  法律関連: {法律関連} 語")
print(f"  条例関連: {条例関連} 語")
print(f"  区域・地域: {区域地域} 語")
print(f"  設備・施設: {設備施設} 語")
print(f"  手続き関連: {手続き} 語")

# ランダムサンプル
print(f"\n【用語サンプル(ランダム60語)】")
sample_size = min(60, len(clean_final_terms))
sample_terms = random.sample(sorted(clean_final_terms), sample_size)
for i, term in enumerate(sorted(sample_terms), 1):
    if i % 3 == 1:
        print(f"  {i:2d}. {term:<25}", end="")
    elif i % 3 == 2:
        print(f" {i:2d}. {term:<25}", end="")
    else:
        print(f" {i:2d}. {term}")
if sample_size % 3 != 0:
    print()

print(f"\n{'=' * 75}")
print(f"  処理が正常に完了しました!")
print(f"  このCSVファイルをMeCabのユーザー辞書として使用できます。")
print(f"{'=' * 75}\n")

final_clean_output


  ✓ MeCab用ユーザー辞書CSVファイルの生成が完了しました

【出力情報】
  ファイル名: /home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv
  登録用語数: 319 語

【入力データ】
  CSVファイル: main2.2.csv
  処理した条文数: 102 件
    ├ 条例: 53 件
    └ 施行規則: 49 件
  総文字数: 523,040 文字

【処理統計】
  初期抽出候補: 872 語
  → フィルタリング後: 745 語
  → 詳細精査後: 809 語
  → 最終精査後: 466 語
  → クリーニング後: 368 語
  → さらに精査後: 330 語
  → 最終クリーン版: 319 語 ★

【用語長統計】
  平均: 8.8 文字
  最短: 2 文字
  最長: 30 文字
  中央値: 5 文字

【カテゴリ別用語数】
  法律関連: 50 語
  条例関連: 82 語
  区域・地域: 44 語
  設備・施設: 55 語
  手続き関連: 19 語

【用語サンプル(ランダム60語)】
   1. かつらぎ町太陽光発電設備の設置に関する条例      2. 事前協議                       3. 事業主
   4. 京田辺市条例                     5. 伝統的建造物群保存地区                6. 住民票の写し
   7. 光発電設備の適正な設置等に関する条例         8. 再生可能エネルギー発電事業              9. 処分
  10. 切土                        11. 協定                        12. 同意
  13. 周辺住民説明会                   14. 国立公園                      15. 土砂災害防止法
  16. 地すべり等防止法                  17. 報告                        18. 天然記念物
  19. 太陽光発電施設の設置に係る条例           20. 太陽光発電施設の

'/home/ubuntu/cur/isep/solar_ordinance_userdic_mecab_main2.2_final.csv'

# MeCabユーザー辞書生成プロジェクト - 完了

## 概要
このノートブックは、**main2.2.csv**に含まれる102件の条例・施行規則(計523,040文字)から、太陽光発電・再生可能エネルギー関連の専門用語を抽出し、MeCab用のユーザー辞書CSVファイルを生成します。

## 処理フロー
1. **データ読み込み**: CSVから「本文」カラムのテキストを統合
2. **候補抽出**: 正規表現を使って条例特有の用語パターンをマッチング
3. **キュレーション**: 重要な法律名・専門用語を手動リストで追加
4. **多段階フィルタリング**: ノイズとなる一般語・文章断片・不完全な法律名を除去
5. **CSV出力**: MeCab IPA辞書形式(13列)で出力

## 最終結果
- **出力ファイル**: `solar_ordinance_userdic_mecab_main2.2_final.csv`
- **登録用語数**: 319語
- **平均用語長**: 8.8文字
- **用語範囲**: 2〜30文字

## カテゴリ別内訳
- 法律関連: 50語
- 条例関連: 82語
- 区域・地域: 44語
- 設備・施設: 55語
- 手続き関連: 19語

## 使用方法
生成されたCSVファイルは、MeCabのユーザー辞書として直接使用できます。

```bash
# MeCabユーザー辞書のコンパイル
/usr/lib/mecab/mecab-dict-index \
  -d /usr/share/mecab/dic/ipadic \
  -u solar_ordinance.dic \
  -f utf-8 \
  -t utf-8 \
  solar_ordinance_userdic_mecab_main2.2_final.csv

# MeCabで使用
mecab -u solar_ordinance.dic
```